# Tree-Unitary Manifold Dimensions

In [2]:
using DelimitedFiles
using TensorOperations
using IterTools
using LinearAlgebra
using Plots
using LaTeXStrings

In [176]:
matrix=readdlm("tu_examples/treeunitary.txt")

tensor=reshape(matrix,(2,2,2,2,2,2));

We now seek the local dimension of the manifold of tree-unitary tensors around this point. 

We first define a function which is zero when we have a tree-unitary tensor.

In [177]:
function check_unitary(tensor)

    res=zeros(ComplexF64,2,2,2,2,2,2)

    @tensor begin
        res[a,b,c,d,e,f]:=tensor[a,b,c,l,m,n]*conj(tensor[d,e,f,l,m,n])
    end
    diff=reshape(res,(8,8))-Matrix(I,8,8)
    # return sum(abs2,diff)
    return reshape(diff,64)
end

function check_tu1(tensor)

    res=zeros(ComplexF64,2,2,2,2)

    @tensor begin
        res[a,b,c,d]:=tensor[i,a,j,l,b,k]*conj(tensor[i,c,j,l,d,k])
    end
    diff=reshape(res,(4,4))-2*Matrix(I,4,4)
    # return sum(abs2,diff)
    return reshape(diff,16)
end

function check_tu2(tensor)

    res=zeros(ComplexF64,2,2,2,2)

    @tensor begin
        res[a,b,c,d]:=tensor[a,i,j,b,l,k]*conj(tensor[c,i,j,d,l,k])
    end
    diff=reshape(res,(4,4))-2*Matrix(I,4,4)
    # return sum(abs2,diff)
    return reshape(diff,16)
end

function check_tu3(tensor)

    res=zeros(ComplexF64,2,2,2,2)

    @tensor begin
        res[a,b,c,d]:=tensor[i,j,a,l,k,b]*conj(tensor[i,j,c,l,k,d])
    end
    diff=reshape(res,(4,4))-2*Matrix(I,4,4)
    # return sum(abs2,diff)
    return reshape(diff,16)
end

function check_treeunitarity(tensor)
    return vcat(check_unitary(tensor),check_tu3(tensor),check_tu1(tensor),check_tu2(tensor))
end

function check_treeunitarity_matrix(matrix)
    return check_treeunitarity(reshape(matrix,(2,2,2,2,2,2)))
end

check_treeunitarity_matrix (generic function with 1 method)

In [178]:
println(check_unitary(tensor))
println(check_tu1(tensor))
println(check_tu2(tensor))
println(check_tu3(tensor))
println(check_treeunitarity(tensor))
println(check_treeunitarity_matrix(matrix))
println(length(check_treeunitarity_matrix(matrix)))

[2.220446049250313e-15, 1.591358063387522e-16, -5.947815812832401e-16, 1.5148128376729303e-16, 3.785042845099404e-16, -3.7555006096193774e-17, 4.803413994540474e-16, 1.0844995837195869e-15, 1.591358063387522e-16, 1.5543122344752192e-15, -7.300264165033465e-17, -8.193974776043665e-16, 7.274190560455419e-16, -1.6155348437310466e-15, -3.1760241719158934e-16, 1.842389609390526e-15, -5.947815812832401e-16, -7.300264165033465e-17, 1.9984014443252818e-15, 1.0500879303953913e-15, 1.2292696274725974e-15, -1.5455337545707617e-16, 2.9776066847233926e-16, 1.044847353896259e-15, 1.5148128376729303e-16, -8.193974776043665e-16, 1.0500879303953913e-15, -6.661338147750939e-16, -2.0825927882886266e-16, 1.9402073166718965e-15, -5.722451990654167e-17, 1.5065761803321771e-15, 3.785042845099404e-16, 7.274190560455419e-16, 1.2292696274725974e-15, -2.0825927882886266e-16, -1.9984014443252818e-15, -4.715245548605644e-16, 3.1959907218716274e-16, 1.127141253818104e-15, -3.7555006096193774e-17, -1.615534843731046

Next, we want to find the numerical derivative of check_treeunitarity with respect to all elements of tensor. 

In [156]:
function jacobian(f, M; h=1e-6)
    M = copy(M)
    sz = size(M)
    N  = length(M)

    # evaluate once to size things
    y0 = vec(f(M))
    p  = length(y0)
    J  = zeros(ComplexF64, p, 2N)

    LI = transpose(LinearIndices(M))

    for j in 1:sz[2], i in 1:sz[1]
        k = LI[i,j]
        x = M[i,j]

        # derivative w.r.t Re(M[i,j])
        M[i,j] = x + h
        fp = vec(f(M))
        M[i,j] = x - h
        fm = vec(f(M))
        J[:, k] = (fp .- fm) ./ (2h)
        M[i,j] = x

        # derivative w.r.t Im(M[i,j])
        M[i,j] = x + im*h
        fp = vec(f(M))
        M[i,j] = x - im*h
        fm = vec(f(M))
        J[:, N+k] = (fp .- fm) ./ (2h)
        M[i,j] = x
    end

    return J
end


jacobian (generic function with 1 method)

In [179]:
# Example: complex-vector output
f(M)=check_treeunitarity_matrix(M)
M = ComplexF64.(matrix)

J = jacobian(f, M)


112×128 Matrix{ComplexF64}:
 -0.204396+0.0im  -0.418042+0.0im  …  0.0+0.0im       0.0+0.0im
  0.379389+0.0im   0.064879+0.0im     0.0+0.0im       0.0+0.0im
 -0.107641+0.0im    0.39864+0.0im     0.0+0.0im       0.0+0.0im
 0.0562748+0.0im   0.774251+0.0im     0.0+0.0im       0.0+0.0im
  0.588755+0.0im   0.196194+0.0im     0.0+0.0im       0.0+0.0im
  0.119502+0.0im  0.0692724+0.0im  …  0.0+0.0im       0.0+0.0im
 -0.620106+0.0im  0.0979546+0.0im     0.0+0.0im       0.0+0.0im
  0.292269+0.0im  -0.375282+0.0im     0.0+0.121759im  0.0-0.182314im
  0.379389+0.0im   0.064879+0.0im     0.0+0.0im       0.0+0.0im
       0.0+0.0im        0.0+0.0im     0.0+0.0im       0.0+0.0im
          ⋮                        ⋱                  
       0.0+0.0im        0.0+0.0im     0.0+0.400345im  0.0+0.00450159im
 -0.209021+0.0im  -0.102198+0.0im     0.0+0.0im       0.0+0.0im
       0.0+0.0im   0.379389+0.0im  …  0.0+0.348432im  0.0+0.0im
       0.0+0.0im  -0.418042+0.0im     0.0+0.0im       0.0+0.0im
       0.

In [180]:
128-rank(conj(J),rtol=1e-9)

37

We can check that this is a good tolerance to use by looking explicitly at the singular values:

In [181]:
println(svdvals(J))

[5.291502622223559, 4.593313984311383, 4.497981058556231, 4.473456378845613, 4.437264430542921, 4.3712246975028695, 4.266511399752278, 4.245749504653271, 4.167682255097111, 4.085462373034352, 4.047161769191299, 4.002349677886473, 3.870050778497821, 3.829976083347316, 3.8025523195865327, 3.690333244670453, 3.618248233084171, 3.532171839737876, 3.4860669296555837, 3.477721761536249, 3.4567326088314494, 3.305478282329591, 3.1690176360525624, 3.154187618793548, 3.0886909690475894, 3.085488145395011, 2.994423071345162, 2.9924193389887943, 2.94538160895403, 2.7232894022978136, 2.7076876325846797, 2.6985600787135535, 2.5523120820864498, 2.51236032867365, 2.460807565774817, 2.440322901463182, 2.424967669384649, 2.3196669597477877, 2.2424973553406438, 2.2055206961757965, 2.1978604950453384, 2.194133847889448, 2.1409111680692945, 2.112928925234982, 2.0812477364585056, 2.057135916784471, 2.000000000016427, 2.0000000000073865, 2.0000000000045017, 2.0000000000007008, 2.000000000000001, 2.0000000000